In [2]:
#list of change,unchanged lines
def sdiffer(s1, s2):
    differ = difflib.Differ()
    diffs = list(differ.compare(s1, s2))

    i = 0
    sdiffs = []
    length = len(diffs)
    while i < length:
        line = diffs[i][2:]
        if diffs[i].startswith('  '):
            sdiffs.append(('u', line))

        elif diffs[i].startswith('+ '):
            sdiffs.append(('+', line))

        elif diffs[i].startswith('- '):
            if i+1 < length and diffs[i+1].startswith('? '): # then diffs[i+2] starts with ('+ '), obviously
                sdiffs.append(('c', line))
                i += 3 if i + 3 < length and diffs[i + 3].startswith('? ') else 2

            elif diffs[i+1].startswith('+ ') and i+2<length and diffs[i+2].startswith('? '):
                sdiffs.append(('c', line))
                i += 2
            else:
                sdiffs.append(('-', line))
        i += 1
    print(sdiffs)
    
    str = '\n'.join(x for x in diffs)
    print(str)

        
#sdiffer(text1_lines, text2_lines)

In [1]:
import difflib

text1 = """typedef int Node, Hash;
    void HashPrint(Hash* hash, void (*PrintFunc)(char*, char*))
    {
        //this is a comment
        //this is a comment
        unsigned int i = 5;
        if (hash == NULL || hash->heads == NULL)
            return;
         //this is a comment
        while(i < hash->table_size)
            Node* temp = hash->heads[i];
            while (temp != NULL)
            {
            
                PrintFunc(temp->entry->key, temp->entry->value);
                temp = temp->next;
            }
    }"""
text1_lines = text1.splitlines()

text2 = """typedef int node, hishchanges;
    void HashPrint(Hash* hash, void (*PrintFunc)(char*, char*))
    {
        unsigned int i1 = 50;
        if (hash == NULL || hash->heads == NULL)
            return;
        for (i = 0; i < hash->table_size; ++i)
        {
            Node* tempo = hash->heads[i];
            while (temp != NULL)
            {
                PrintFunc(temp->entry->key, temp->entry->value);
                temp = temp->next;
            }
        }
    }"""
text2_lines = text2.splitlines()

#function to find out the range of difference
def getRange(li,stri): 
    #print(str)
    finallist = []
    #addedtokens = []
    seed = li[0]
    prev= li[0]
    range = 1
    epoch = 0
    rangesum = 0
    for i in li:
        diff = i-prev
        if diff == 1:
            range += 1
        elif diff != 0:
            
            k = stri[:seed].rfind(' ')
            k1 = stri[(seed + range):].find(' ')
            new_string = ''
            if k1 != -1:
                new_string = stri[k+1:(k1+seed+range)]
            else:
                new_string = stri[k+1:]
            
            finallist.append([seed - (epoch*rangesum),range , stri[seed :seed + range]]) #new_string])
            #addedtokens.append(stri[seed :seed + range])
            seed = i
            rangesum += range
            epoch = 1
            range = 1
        prev = i
      
    k = stri[:seed].rfind(' ')
    k1 = stri[(seed + range):].find(' ')
    new_string = ''
    if k1 != -1:
        new_string = stri[k+1:(k1+seed+range)]
    else:
        new_string = stri[k+1:]
        
    finallist.append([seed - (epoch*rangesum),range , stri[seed :seed + range]]) #new_string])
    #addedtokens.append(stri[seed :seed + range])
    return finallist#,addedtokens
        
#print only the changed line with corresponding line number
def printdiffs(diff):
    lineno=0
    newlineno = 0
    prevline = ''
    lengthdiff = sum(1 for i in diff)
    replacedtoken = []
    for diffindex in range(len(diff)):
        
        flag = 0
        i = diff[diffindex]
            
        #line number count
        if i.startswith(' ') or i.startswith('-'):
            lineno += 1
            newlineno += 1
            
        if i.startswith('+'): # whole line change - insertion
            if diffindex+1<(len(diff)):
                next_line = diff[diffindex+1]
                if next_line.startswith('?') == False and prevline.startswith('?') == False: #in line changes must be followed or lead by ?
                    newlineno += 1
                    flag = 1
                    print('Insertion in line '+str(newlineno) +':' + (str(i[2:])))
            else: # last line insertion
                if prevline.startswith('?') == False: #in line changes must be followed or lead by ?
                    newlineno += 1
                    flag = 1
                    print('Insertion in line '+str(newlineno) +':' + (str(i[2:])))
                
       # if i.startswith(' ') == False: ##show only the changed lines in the output
        #    if flag == 1:
         #       print(str(lineno+1) + ' : ' + i)
         #   else:
          #      print(str(lineno) + ' : ' + i)
            
        if i.startswith('-'): # whole line change - deletion
            if diffindex+1<(len(diff)):
                next_line = diff[diffindex+1]
                if next_line.startswith(' ') or next_line.startswith('-'): #update is followed by ? & new insertion is followed by +, remove otherwise 
                    print('deletion in line '+str(lineno) +':' + (str(i[2:])))
                    newlineno -= 1
                elif  next_line.startswith('+'): #we need to check the next line too to see if this is an update
                    if diffindex+2 < len(diff):
                        isitachangeline =  diff[diffindex+2]
                        if isitachangeline.startswith('?') == False:
                            print('deletion in line '+str(lineno) +':' + (str(i[2:])))
                            newlineno -= 1
                    else:
                        print('deletion in line '+str(lineno) +':' + (str(i[2:])))
                        newlineno -= 1
            else: #last line
                #newlineno -= 1
                print('deletion in line '+str(newlineno) +':' + (str(i[2:])))
        
        if i.startswith('?'):
            li = [index for index, ltr in enumerate(i[2:]) if ltr == '^']
            if len(li):
                li = getRange(li,prevline[2:]) #previous line of code to retrieve the changes values
                if prevline.startswith('-'):
                    replacedtoken = li #store the tokens before any update
                else:
                    #print('replace :' + (str(li)))
                    #print('replaced tokens :' + (str(replacedtoken)))
                    #replacedtoken2 = copy.deepcopy(replacedtoken)
                    for temporary_Iterator in range(len(li)):
                        replacedtoken[temporary_Iterator].append(li[temporary_Iterator][-1]) # map the changed and unchanged tokens
                    print('replaced tokens in line '+str(lineno) +':'+ (str(replacedtoken))) 

            li = [index for index, ltr in enumerate(i[2:]) if ltr == '-']
            if len(li):
                li = getRange(li,prevline[2:])
                print('deleted tokens in line '+str(lineno) +':'+ (str(li)))
                #print('deleted tokens :' + (str(addedtokens)))

            li = [index for index, ltr in enumerate(i[2:]) if ltr == '+']
            if len(li):    
                #print('insertion :' + (str(li)))
                li = getRange(li,prevline[2:])
                print('inserted tokens in line '+str(lineno) +':'+  (str(li)))
                #print('inserted tokens :' + (str(addedtokens)))
        prevline = i        
        
                
d = difflib.Differ()
diff = d.compare(text1_lines, text2_lines)
#print('\n'.join(diff))#, end="")

#delta = '\n'.join(x[2:] for x in diff if x.startswith('- '))
#print(delta)

printdiffs(list(diff)) #convert generator to list


#shows changed indexed as well

replaced tokens in line 1:[[12, 1, 'N', 'n'], [17, 2, 'Ha', 'hi']]
inserted tokens in line 1:[[22, 7, 'changes']]
deletion in line 4:        //this is a comment
deletion in line 5:        //this is a comment
inserted tokens in line 6:[[22, 1, '1'], [26, 1, '0']]
deletion in line 9:         //this is a comment
deletion in line 10:        while(i < hash->table_size)
Insertion in line 7:        for (i = 0; i < hash->table_size; ++i)
Insertion in line 8:        {
inserted tokens in line 11:[[22, 1, 'o']]
deletion in line 14:            
Insertion in line 15:        }
